In [8]:
!git clone https://github.com/divya-r-kamat/seq2py.git

Cloning into 'seq2py'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [9]:
!git config --global user.email 'divya.r.kamat@gmail.com'
!git config --global user.name 'divya-r-kamat'

In [23]:
!ls

 conala-corpus-v1.1.zip   Translate_English_text_to_python_code_baseline.ipynb
 data			  Translate_English_text_to_python_training.ipynb
 model			  tut6-model.pt
 seq2py_inference.ipynb   utils
'seq2py webapp.ipynb'


In [21]:
cd /content/drive/MyDrive/seq2py

/content/drive/MyDrive/seq2py


In [26]:
!ls /content/seq2py/

README.md


In [27]:
cp /content/seq2py/README.md /content/drive/MyDrive/seq2py/.

In [29]:
!git 

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [40]:
!pip install -U ipykernel

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.7/dist-packages (5.5.0)


In [41]:
!pip install -q streamlit

In [42]:
!pip install pyngrok

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%writefile score.py

import torch
from torchtext import data 
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import unicodedata
import codecs
import csv
import time
import random
import re
import os
import pickle
from io import open
import itertools
import math
import pandas as pd
from torchtext.data import Field, BucketIterator, LabelField, TabularDataset
import streamlit as st

BASE_DIR='/content/drive/MyDrive/seq2py'
#https://stackoverflow.com/questions/62922640/calling-a-function-in-a-different-python-file-using-google-colab
%cd $BASE_DIR/utils
import preprocess, helper,transformer
from transformer import Encoder,Decoder,Seq2Seq

st.title("Generate python code from english text")

with open(os.path.join(BASE_DIR ,"model/src_stio.pkl"),"rb") as f:
  src_stoi = pickle.load(f)
with open(os.path.join(BASE_DIR ,"model/src_itos.pkl"),"rb") as f:
  src_itos = pickle.load(f)


with open(os.path.join(BASE_DIR ,"model/trg_stio.pkl"),"rb") as f:
  trg_stoi = pickle.load(f)
with open(os.path.join(BASE_DIR ,"model/trg_itos.pkl"),"rb") as f:
  trg_itos = pickle.load(f)

  INPUT_DIM = len(src_stoi)
OUTPUT_DIM = len(trg_stoi)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.25

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)


dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT,
              device)


SRC_PAD_IDX = src_stoi['<pad>']
TRG_PAD_IDX = trg_stoi['<pad>']

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

@st.cache
def load_model(model_path):
    # Load model
    trained_model = os.path.join(BASE_DIR,"model/tut6-model.pt")
    with st.spinner("Loading model into memory"):
      model.load_state_dict(torch.load(trained_model));
    model.eval()
    return model

sentence = st.text_input("Enter english text:")
if sentence is not None:
            # Normalize sentence
            # input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
      output_words,_ = helper.translate_sentence(input_sentence,src_field,trg_field,trg_itos, model, device)
      # Format and print response sentence
      output_words[:] = [x for x in output_words if not (x == '<eos>' or x == '<pad>')]
      st.write('Python Code Generated:\n', ' '.join(output_words))


Overwriting score.py


In [45]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [3]:
cat nohup.out


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.202.212.58:8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8502
  External URL: http://35.202.212.58:8502

  Stopping...
  Stopping...

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.202.212.58:8501



In [65]:
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=3fa8d7fba98e521d095fc584ac2fc59dbf60b135e41c98ce8d5f9837bb522dfd
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok
  Found existing installation: pyngrok 5.0.4
    Uninstalling pyngrok-5.0.4:
      Successfully uninstalled pyngrok-5.0.4


In [34]:
#Reference

# https://medium.com/@jcharistech/how-to-run-streamlit-apps-from-colab-29b969a1bdfc#:~:text=In%20case%20you%20want%20to,and%20then%20kill%20that%20process.
# !ps aux | grep streamlit
# !kill
# ngrok.get_tunnels()
# ngrok.disconnect('http://8ec42087e0e0.ngrok.io')

root         513  3.7  0.8 582908 118508 ?       Sl   07:34   2:49 /usr/bin/python3 /usr/local/bin/streamlit run score.py
root         724  4.0  0.8 582908 118476 ?       Sl   08:39   0:27 /usr/bin/python3 /usr/local/bin/streamlit run score.py
root         855  0.0  0.0  39204  6544 ?        S    08:51   0:00 /bin/bash -c ps aux | grep streamlit
root         857  0.0  0.0  38576  5516 ?        S    08:51   0:00 grep streamlit


In [5]:
from pyngrok import ngrok

ngrok.get_tunnels()

[]

In [61]:
ngrok.disconnect('http://47636ea30962.ngrok.io')

In [6]:
from pyngrok import ngrok
ngrok.set_auth_token("1pVsWrMYOHo5XZccFlZ8N6qejUh_4Jngiuw6qSk89v94wPyCG")
url = ngrok.connect(port=8501)
url

'http://a53be42846bf.ngrok.io'

In [63]:
!cat nohup.out


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.202.212.58:8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8502
  External URL: http://35.202.212.58:8502

  Stopping...
  Stopping...

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.202.212.58:8501

